## Creating a sqlite database with Python
<p>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/SQLite370.svg/440px-SQLite370.svg.png" width=230 align="left" padding=2>
SQLite is a relational database management system based on the SQL language optimized for use in small environments such as mobile apps. It does not require a separate server process to be run as needed in large database engines such as MySQL and Oracle. It's integrated with Python using a module called sqlite3.

In [ ]:
import sqlite3
from pathlib import Path

### Create a sqlite database
We can start by creating the database file `student.db`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
## finding path
# !ls drive/MyDrive/SJSU/SJSU_Fall2024_CS/CS133_Data-Visualization/week10_sql


Colab_Lec14_Python_Relational_DB_SQLite.ipynb  Lec14_sql.gslides


In [ ]:
gdrive='drive/MyDrive/SJSU/SJSU_Fall2024_CS/CS133_Data-Visualization/week10_sql/'
Path(gdrive+'student.db').touch()

First, we will create a database connection and a cursor to execute SQL commands.

## Define the relationships between your data

Let's create a database to store the information about students and the courses that they enroll in.

- The `student` table that stores student information.
- The `course` table that stores course information.
- The `enrollment` table that stores the relationship between students and courses.

The following Entity Relationship Diagram (ERD) illustrates tables:student, course, and enrollment.
You can create it in Lucidchart (SJSU has license for students at SJSU). You can find it in [one.sjsu.edu](https://one.sjsu.edu)

<img src="https://www.sjsu.edu/people/wendy.lee/pics/CS122/ERD_school.png" width=450>
<br>
<img src="https://www.sjsu.edu/people/wendy.lee/pics/CS122/ERD_Cardinality.png" width=300>


You can export the SQL command from the Lucidchart ERD diagram tool but you will need to modify it to be accepted by SQLite.

`CREATE TABLE student (
  student_id INTEGER,
  firstname TEXT,
  lastname TEXT,
  PRIMARY KEY (student_id)
);`

`CREATE TABLE enrollment (
  student_id INTEGER,
  course_id INTEGER,
  term TEXT,
  year INTEGER
);`

`CREATE TABLE course (
  course_id INTEGER,
  name TEXT,
  desc TEXT,
  PRIMARY KEY (course_id)
);`

#### [Data types in SQLite](https://www.sqlite.org/datatype3.html)
There are 5 data types for values stored in in an SQLite database:

- NULL. The value is a NULL value.

- INTEGER. The value is a signed integer, stored in 1, 2, 3, 4, 6, or 8 bytes depending on the magnitude of the value.

- REAL. The value is a floating point value, stored as an 8-byte IEEE floating point number.

- TEXT. The value is a text string, stored using the database encoding (UTF-8, UTF-16BE or UTF-16LE).

- BLOB. The value is a blob of data, stored exactly as it was input.

[Learn more](https://www.guru99.com/database-normalization.html) about designing your relationship database tables to eliminate data redundancy and improve data integrity.

In [ ]:
conn = sqlite3.connect(gdrive+'student.db') # creates a connection to the database
c = conn.cursor() # the cursor is what we'll actually use

### Execute a SQL command to create a `student` table with 3 columns `student_id`, `firstname`, `lastname`.
Here we set the id as a primary key.

In [ ]:
# Let's create a student table
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS student (
             student_id INTEGER PRIMARY KEY,
             firstname TEXT NOT NULL,
             lastname TEXT NOT NULL
             )'''

c.execute(SQL_CreateTable)

In [ ]:
# Add data
SQL_InsertStmt = """INSERT INTO student VALUES
                 (1284729,'Wendy', 'Lee'),
                 (1013832,'Martin', 'Jones')"""

c.execute(SQL_InsertStmt)

In [ ]:
c.execute("SELECT * FROM student WHERE firstname='Wendy'")
results = c.fetchall()
print(results)

[(1284729, 'Wendy', 'Lee')]


#### Let's create the other two tables: `course` and `enrollment`.

In [ ]:
SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS course (
             course_id INTEGER PRIMARY KEY,
             name TEXT NOT NULL,
             desc TEXT NOT NULL
             )'''

c.execute(SQL_CreateTable)

In [ ]:
#PRAGMA foreign_keys=ON;

SQL_CreateTable = '''CREATE TABLE IF NOT EXISTS enrollment (
             student_id INTEGER,
             course_id INTEGER,
             term TEXT NOT NULL,
             year INTEGER,
             FOREIGN KEY (student_id)
                 REFERENCES student (student_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION,
             FOREIGN KEY (course_id)
                 REFERENCES course (course_id)
                     ON DELETE CASCADE
                     ON UPDATE NO ACTION
             )'''

c.execute(SQL_CreateTable)

`DELETE CASCADE`: When we create a foreign key using this option, it deletes the referencing rows in the child table when the referenced row is deleted in the parent table which has a primary key.

`UPDATE NO ACTION`: When we create a foreign key using this option, the foreign key constraint (NO ACTION) behaves whenever the parent key is updated.

Read more about [foreign keys](https://www.sqlite.org/foreignkeys.html) in SQLite.

## Load CSV file into sqlite table

In [ ]:
import pandas as pd

# load the course data into panda dataframe
course = pd.read_csv("https://raw.githubusercontent.com/csbfx/advpy122-data/master/course.csv")
course

,course_id,name,desc
0,28732,CS122,Advanced Python
1,24852,CS22B,Intro to Python II
2,37473,CS22A,Intro to Python I


In [ ]:
# insert the data from dataframe to database table `course`
course.to_sql('course', conn, if_exists='append', index=False)

In [ ]:
c.execute("SELECT * FROM course")
results = c.fetchall()
print(results)

[(24852, 'CS22B', 'Intro to Python II'), (28732, 'CS122', 'Advanced Python'), (37473, 'CS22A', 'Intro to Python I')]


In [ ]:
# load the enrollment data into panda dataframe
edf = pd.read_csv("https://raw.githubusercontent.com/csbfx/advpy122-data/master/enroll.csv")
edf

,student_id,course_id,term,year
0,1284729,24852,Fall,2020
1,1013832,28732,Spring,2021


In [ ]:
# insert the data from dataframe to database table `enrollment`
edf.to_sql('enrollment', conn, if_exists='append', index=False)

2

In [ ]:
c.execute("SELECT * FROM enrollment")
results = c.fetchall()
print(results)

[(1284729, 24852, 'Fall', 2020), (1013832, 28732, 'Spring', 2021)]


In [ ]:
# Can read directly from a SQL Query to a pandas dataframe
# https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html
pd.read_sql_query("SELECT * FROM enrollment", conn)

,student_id,course_id,term,year
0,1284729,24852,Fall,2020
1,1013832,28732,Spring,2021


In [ ]:
SQL_JointQuery = """SELECT student.firstname, student.lastname,
                course.name FROM student
                JOIN enrollment using (student_id)
                JOIN course using (course_id)
                WHERE student.lastname='Lee'
                """
c.execute(SQL_JointQuery)
joint_results = c.fetchall()
print(joint_results)

[('Wendy', 'Lee', 'CS22B')]


In [ ]:
# Another way to join tables
SQL_JointQuery2 = """SELECT student.firstname, student.lastname,
                course.name FROM student
                JOIN enrollment ON enrollment.student_id = student.student_id
                JOIN course ON enrollment.course_id = course.course_id
                WHERE student.lastname='Lee'
                """
c.execute(SQL_JointQuery2)
joint_results2 = c.fetchall()
print(joint_results2)

[('Wendy', 'Lee', 'CS22B')]


In [ ]:
conn.commit() # commit the current transaction
# cursor makes changes, connection commits
conn.close() # close the connection